In [83]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_samples, silhouette_score
import pandas as pd
import numpy as np
import plotly.express as px

# Pandas Config
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [84]:
def reverse_ohe(df, features, ignoredFeatures, featuresLength, prefix, suffix = ''):
    all_features = ignoredFeatures + features
    df_pivot = pd.DataFrame(columns = all_features)

    for i in range(featuresLength):
        cols_aux = [f'{feature}{prefix}{i}{suffix}' for feature in features]
        df_aux = df[ignoredFeatures + cols_aux].copy()
        df_aux.columns = all_features
        df_pivot = pd.concat([df_pivot, df_aux])

    return df_pivot.sort_values(by='Data_Hora').copy()

In [85]:
merged = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/merged.csv',
                sep=';',
                dtype = {'Local_0': object, 'Local_1':object,
                          'Local_2':object,  'Local_3':object})

merged.head()

,index,Data,Hora,Data_Hora,UmidadeRelativa_0,PressaoAtmosferica_0,TemperaturaDoAr_0,TemperaturaInterna_0,PontoDeOrvalho_0,SensacaoTermica_0,RadiacaoSolar_0,DirecaoDoVento_0,VelocidadeDoVento_0,Precipitacao_0,Local_0,UmidadeRelativa_1,PressaoAtmosferica_1,TemperaturaDoAr_1,TemperaturaInterna_1,PontoDeOrvalho_1,SensacaoTermica_1,RadiacaoSolar_1,DirecaoDoVento_1,VelocidadeDoVento_1,Precipitacao_1,Local_1,UmidadeRelativa_2,PressaoAtmosferica_2,TemperaturaDoAr_2,TemperaturaInterna_2,PontoDeOrvalho_2,SensacaoTermica_2,RadiacaoSolar_2,DirecaoDoVento_2,VelocidadeDoVento_2,Precipitacao_2,Local_2,UmidadeRelativa_3,PressaoAtmosferica_3,TemperaturaDoAr_3,TemperaturaInterna_3,PontoDeOrvalho_3,SensacaoTermica_3,RadiacaoSolar_3,DirecaoDoVento_3,VelocidadeDoVento_3,Precipitacao_3,Local_3,UmidadeRelativa_4,PressaoAtmosferica_4,TemperaturaDoAr_4,TemperaturaInterna_4,PontoDeOrvalho_4,SensacaoTermica_4,RadiacaoSolar_4,DirecaoDoVento_4,VelocidadeDoVento_4,Precipitacao_4,Local_4
0,26,01/01/11,06:30:00,2011-01-01 06:30:00,87.2,918.6,19.5,22.6,17.3,NaN,8.0,163.0,1.1,0.0,Camilopolis,89.2,918.7,19.1,22.6,17.3,NaN,9.0,109.0,2.7,0.0,Erasmo,89.4,917.2,19.1,22.5,17.3,NaN,8.0,113.0,2.8,0.0,Paraiso,86.5,924.8,19.7,23.2,17.4,NaN,8.0,149.0,1.2,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
1,27,01/01/11,06:45:00,2011-01-01 06:45:00,86.6,918.7,19.5,22.5,17.2,NaN,18.0,160.0,1.4,0.0,Camilopolis,88.6,918.7,19.2,22.6,17.3,NaN,16.0,127.0,2.0,0.0,Erasmo,89.1,917.2,19.1,22.5,17.3,NaN,16.0,154.0,1.6,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,16.0,162.0,1.8,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
2,28,01/01/11,07:00:00,2011-01-01 07:00:00,86.0,918.7,19.6,22.6,17.2,NaN,41.0,108.0,2.2,0.0,Camilopolis,88.1,918.7,19.3,22.7,17.3,NaN,35.0,138.0,2.7,0.0,Erasmo,88.4,917.2,19.1,22.6,17.3,NaN,28.0,116.0,2.3,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,30.0,181.0,1.7,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
3,29,01/01/11,07:15:00,2011-01-01 07:15:00,85.4,918.7,19.7,22.6,17.2,NaN,62.0,125.0,2.3,0.0,Camilopolis,88.1,918.7,19.3,22.7,17.3,NaN,50.0,135.0,1.4,0.0,Erasmo,89.3,917.3,19.0,22.7,17.2,NaN,50.0,128.0,2.5,0.0,Paraiso,84.4,924.9,20.0,23.3,17.2,NaN,53.0,151.0,1.5,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
4,30,01/01/11,07:30:00,2011-01-01 07:30:00,84.7,918.8,19.8,22.7,17.1,NaN,82.0,146.0,1.8,0.0,Camilopolis,87.4,918.7,19.4,22.8,17.2,NaN,88.0,139.0,3.7,0.0,Erasmo,88.6,917.3,19.0,22.7,17.1,NaN,77.0,120.0,1.6,0.0,Paraiso,84.2,924.9,19.8,23.4,17.1,NaN,66.0,176.0,3.0,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria


In [86]:
merged[['Precipitacao_0', 'Precipitacao_1', 'Precipitacao_2', 'Precipitacao_3', 'Precipitacao_4']].corr()

,Precipitacao_0,Precipitacao_1,Precipitacao_2,Precipitacao_3,Precipitacao_4
Precipitacao_0,1.000000,0.506489,0.476399,0.494459,0.503100
Precipitacao_1,0.506489,1.000000,0.354365,0.364162,0.394172
Precipitacao_2,0.476399,0.354365,1.000000,0.583411,0.612899
Precipitacao_3,0.494459,0.364162,0.583411,1.000000,0.538128
Precipitacao_4,0.503100,0.394172,0.612899,0.538128,1.000000


In [87]:
regions = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/error_regions.csv', sep=';')

regions.head()

,Data_Hora,UmidadeRelativa_0_error,UmidadeRelativa_1_error,UmidadeRelativa_2_error,UmidadeRelativa_3_error,UmidadeRelativa_4_error,PressaoAtmosferica_0_error,PressaoAtmosferica_1_error,PressaoAtmosferica_2_error,PressaoAtmosferica_3_error,PressaoAtmosferica_4_error,TemperaturaDoAr_0_error,TemperaturaDoAr_1_error,TemperaturaDoAr_2_error,TemperaturaDoAr_3_error,TemperaturaDoAr_4_error,TemperaturaInterna_0_error,TemperaturaInterna_1_error,TemperaturaInterna_2_error,TemperaturaInterna_3_error,TemperaturaInterna_4_error,PontoDeOrvalho_0_error,PontoDeOrvalho_1_error,PontoDeOrvalho_2_error,PontoDeOrvalho_3_error,PontoDeOrvalho_4_error,SensacaoTermica_0_error,SensacaoTermica_1_error,SensacaoTermica_2_error,SensacaoTermica_3_error,SensacaoTermica_4_error,RadiacaoSolar_0_error,RadiacaoSolar_1_error,RadiacaoSolar_2_error,RadiacaoSolar_3_error,RadiacaoSolar_4_error,DirecaoDoVento_0_error,DirecaoDoVento_1_error,DirecaoDoVento_2_error,DirecaoDoVento_3_error,DirecaoDoVento_4_error,VelocidadeDoVento_0_error,VelocidadeDoVento_1_error,VelocidadeDoVento_2_error,VelocidadeDoVento_3_error,VelocidadeDoVento_4_error,Precipitacao_0_error,Precipitacao_1_error,Precipitacao_2_error,Precipitacao_3_error,Precipitacao_4_error
0,2011-01-01 00:00:00,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
1,2011-01-01 00:15:00,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
2,2011-01-01 00:30:00,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
3,2011-01-01 00:45:00,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
4,2011-01-01 01:00:00,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,False,True,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False


In [88]:
df = merged.merge(regions, on = 'Data_Hora')

df.head()

,index,Data,Hora,Data_Hora,UmidadeRelativa_0,PressaoAtmosferica_0,TemperaturaDoAr_0,TemperaturaInterna_0,PontoDeOrvalho_0,SensacaoTermica_0,RadiacaoSolar_0,DirecaoDoVento_0,VelocidadeDoVento_0,Precipitacao_0,Local_0,UmidadeRelativa_1,PressaoAtmosferica_1,TemperaturaDoAr_1,TemperaturaInterna_1,PontoDeOrvalho_1,SensacaoTermica_1,RadiacaoSolar_1,DirecaoDoVento_1,VelocidadeDoVento_1,Precipitacao_1,Local_1,UmidadeRelativa_2,PressaoAtmosferica_2,TemperaturaDoAr_2,TemperaturaInterna_2,PontoDeOrvalho_2,SensacaoTermica_2,RadiacaoSolar_2,DirecaoDoVento_2,VelocidadeDoVento_2,Precipitacao_2,Local_2,UmidadeRelativa_3,PressaoAtmosferica_3,TemperaturaDoAr_3,TemperaturaInterna_3,PontoDeOrvalho_3,SensacaoTermica_3,RadiacaoSolar_3,DirecaoDoVento_3,VelocidadeDoVento_3,Precipitacao_3,Local_3,UmidadeRelativa_4,PressaoAtmosferica_4,TemperaturaDoAr_4,TemperaturaInterna_4,PontoDeOrvalho_4,SensacaoTermica_4,RadiacaoSolar_4,DirecaoDoVento_4,VelocidadeDoVento_4,Precipitacao_4,Local_4,UmidadeRelativa_0_error,UmidadeRelativa_1_error,UmidadeRelativa_2_error,UmidadeRelativa_3_error,UmidadeRelativa_4_error,PressaoAtmosferica_0_error,PressaoAtmosferica_1_error,PressaoAtmosferica_2_error,PressaoAtmosferica_3_error,PressaoAtmosferica_4_error,TemperaturaDoAr_0_error,TemperaturaDoAr_1_error,TemperaturaDoAr_2_error,TemperaturaDoAr_3_error,TemperaturaDoAr_4_error,TemperaturaInterna_0_error,TemperaturaInterna_1_error,TemperaturaInterna_2_error,TemperaturaInterna_3_error,TemperaturaInterna_4_error,PontoDeOrvalho_0_error,PontoDeOrvalho_1_error,PontoDeOrvalho_2_error,PontoDeOrvalho_3_error,PontoDeOrvalho_4_error,SensacaoTermica_0_error,SensacaoTermica_1_error,SensacaoTermica_2_error,SensacaoTermica_3_error,SensacaoTermica_4_error,RadiacaoSolar_0_error,RadiacaoSolar_1_error,RadiacaoSolar_2_error,RadiacaoSolar_3_error,RadiacaoSolar_4_error,DirecaoDoVento_0_error,DirecaoDoVento_1_error,DirecaoDoVento_2_error,DirecaoDoVento_3_error,DirecaoDoVento_4_error,VelocidadeDoVento_0_error,VelocidadeDoVento_1_error,VelocidadeDoVento_2_error,VelocidadeDoVento_3_error,VelocidadeDoVento_4_error,Precipitacao_0_error,Precipitacao_1_error,Precipitacao_2_error,Precipitacao_3_error,Precipitacao_4_error
0,26,01/01/11,06:30:00,2011-01-01 06:30:00,87.2,918.6,19.5,22.6,17.3,NaN,8.0,163.0,1.1,0.0,Camilopolis,89.2,918.7,19.1,22.6,17.3,NaN,9.0,109.0,2.7,0.0,Erasmo,89.4,917.2,19.1,22.5,17.3,NaN,8.0,113.0,2.8,0.0,Paraiso,86.5,924.8,19.7,23.2,17.4,NaN,8.0,149.0,1.2,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
1,27,01/01/11,06:45:00,2011-01-01 06:45:00,86.6,918.7,19.5,22.5,17.2,NaN,18.0,160.0,1.4,0.0,Camilopolis,88.6,918.7,19.2,22.6,17.3,NaN,16.0,127.0,2.0,0.0,Erasmo,89.1,917.2,19.1,22.5,17.3,NaN,16.0,154.0,1.6,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,16.0,162.0,1.8,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
2,28,01/01/11,07:00:00,2011-01-01 07:00:00,86.0,918.7,19.6,22.6,17.2,NaN,41.0,108.0,2.2,0.0,Camilopolis,88.1,918.7,19.3,22.7,17.3,NaN,35.0,138.0,2.7,0.0,Erasmo,88.4,917.2,19.1,22.6,17.3,NaN,28.0,116.0,2.3,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,30.0,181.0,1.7,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False
3,29,01/01/11,07:1

In [89]:
orders = pd.read_csv('../../../data/cleandata/Ordens de serviço/Enchentes_LatLong.csv', sep=';')

orders.head()

,ID1,ID2,Hora,Endereco1,Endereco2,Comentario1,Comentario2,Status,Data,lat,lng,status
0,NaN,003.578/2.010,19:53:42,AVN MAURICIO DE MEDEIROS - ASS. MARG. GUARARÁ...,JD IRENE,NaN,NaN,Encerrada,07/01/2010,-23.709969,-46.508143,0.0
1,NaN,003.579/2.010,19:57:47,AVN MAURICIO DE MEDEIROS - ASS. MARG. GUARARÁ...,JD IRENE,NaN,RG - 40490344-7,Encerrada,07/01/2010,-23.709969,-46.508143,0.0
2,16219035.0,003.595/2.010,21:35:20,"RUA ORATORIO, 5081 MAZUCO MADEIREIRA",BANGU,Comercio fechado Não havia pessoas para abrir ...,NAS PARTES DA ORATORIO.HOUVE ENCHENTENA PARTE ...,Encerrada,07/01/2010,-23.625805,-46.492110,0.0
3,6157056.0,003.597/2.010,21:39:58,"AVN ESTADOS, DOS, 5911 MAZUCO MADEIREIRA",SANTA TEREZINHA,NaN,"ALAGAMENTO,PERDA DE MATERIAL > AZULEJOS - Ope...",Encerrada,07/01/2010,-23.650780,-46.521617,0.0
4,5076139.0,003.536/2.010,17:33:30,"RUA SEN FLAQUER, 935",CASA BRANCA,NaN,fabrica alagou - Operador: JOSEROS,Encerrada,07/01/2010,-23.665427,-46.519736,0.0


In [90]:
orders[orders['Data'] == '07/01/2011']

,ID1,ID2,Hora,Endereco1,Endereco2,Comentario1,Comentario2,Status,Data,lat,lng,status
150,4021007.0,002.940/2.011,20:12:17,"RUA ORATORIO, 4641 ESQUINA RUA ALMADA",BANGU,O ESTABELECIMENTO PERMANECIA FECHADO. TENTATIV...,COMERCIO DE PEÇAS KAPITAL.MAIS DE 1 M. DE AGUA...,Encerrada,07/01/2011,-23.625900,-46.496609,0.0
151,16219035.0,002.943/2.011,20:41:47,"RUA ORATORIO, 5081 MAZUCO:MADEREIRA",BANGU,NaN,"RIBEIRÃO DO ORATORIO, FUNDOS DA LOJA COM PERD...",Encerrada,07/01/2011,-23.625805,-46.492110,0.0
152,11115067.0,002.892/2.011,17:18:07,"RUA JACINTOS, DOS, 454 E FDS",JD GUARARA,DMO-SEMASA,O SEMASA FEZ UM SERVIÇO NA CALCADA ENTRADA DE ...,Encerrada,07/01/2011,-23.703319,-46.510162,0.0


In [91]:
df[df['Data'] == '07/01/11'][['Hora', 'Precipitacao_0']]

,Hora,Precipitacao_0
7991,06:45:00,0.0
7992,07:00:00,0.0
7993,07:15:00,0.0
7994,07:30:00,0.0
7995,07:45:00,0.0
7996,08:00:00,0.0
7997,08:15:00,0.0
7998,08:30:00,0.0
7999,08:45:00,0.0
8000,09:00:00,0.0


In [92]:
orders[orders['Data'] == '15/04/2018']

,ID1,ID2,Hora,Endereco1,Endereco2,Comentario1,Comentario2,Status,Data,lat,lng,status
761,11135013.0,060.695/2.018,15:57:58,"RUA COCAIS, DOS, 640 FDS.",VILA LUZITA,dmo(psa),usuario inf. alagamento dentr de casa - Operad...,Encerrada,15/04/2018,-23.703972,-46.506439,0.0


In [93]:
df[df['Data'] == '15/04/18'][['Hora', 'Precipitacao_0']]

,Hora,Precipitacao_0
87208,06:30:00,0.4
87209,06:45:00,0.2
87210,07:00:00,0.0
87211,07:15:00,0.0
87212,07:30:00,0.0
87213,07:45:00,0.0
87214,08:00:00,0.0
87215,08:15:00,0.0
87216,08:30:00,0.2
87217,08:45:00,0.6


In [94]:
features = [
    'Local',
    'UmidadeRelativa',
    'PressaoAtmosferica',
    'TemperaturaDoAr',
    'TemperaturaInterna',
    'PontoDeOrvalho',
    'SensacaoTermica',
    'RadiacaoSolar',
    'DirecaoDoVento',
    'VelocidadeDoVento',
    'Precipitacao',
]

ignoredFeatures = [
    'Data_Hora'
]

In [95]:
repaired = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/repaired.csv', sep=';')

repaired.head()

,Data_Hora,RadiacaoSolar_2_interpol,DirecaoDoVento_2_interpol,DirecaoDoVento_3_interpol,RadiacaoSolar_3_interpol,RadiacaoSolar_0_interpol,DirecaoDoVento_0_interpol,PressaoAtmosferica_2_interpol,PressaoAtmosferica_3_interpol,PressaoAtmosferica_0_interpol,VelocidadeDoVento_3_interpol,PressaoAtmosferica_4_interpol,VelocidadeDoVento_2_interpol,TemperaturaDoAr_3_interpol,TemperaturaDoAr_2_interpol,VelocidadeDoVento_0_interpol,TemperaturaDoAr_0_interpol,UmidadeRelativa_3_interpol,PontoDeOrvalho_3_interpol,TemperaturaDoAr_4_interpol,TemperaturaInterna_3_interpol,UmidadeRelativa_0_interpol,TemperaturaInterna_2_interpol,TemperaturaInterna_0_interpol,PontoDeOrvalho_0_interpol,TemperaturaInterna_4_interpol,TemperaturaDoAr_1_interpol,VelocidadeDoVento_4_interpol,PontoDeOrvalho_2_interpol,UmidadeRelativa_1_interpol,PontoDeOrvalho_1_interpol,TemperaturaInterna_1_interpol,UmidadeRelativa_2_interpol,RadiacaoSolar_4_interpol,DirecaoDoVento_4_interpol,PontoDeOrvalho_4_interpol,DirecaoDoVento_1_interpol,UmidadeRelativa_4_interpol,PressaoAtmosferica_1_interpol,VelocidadeDoVento_1_interpol,SensacaoTermica_0_interpol,SensacaoTermica_2_interpol,SensacaoTermica_3_interpol,RadiacaoSolar_1_interpol,SensacaoTermica_4_interpol,SensacaoTermica_1_interpol,RadiacaoSolar_2_pred,RadiacaoSolar_2_repaired,DirecaoDoVento_2_pred,DirecaoDoVento_2_repaired,DirecaoDoVento_3_pred,DirecaoDoVento_3_repaired,RadiacaoSolar_3_pred,RadiacaoSolar_3_repaired,RadiacaoSolar_0_pred,RadiacaoSolar_0_repaired,DirecaoDoVento_0_pred,DirecaoDoVento_0_repaired,PressaoAtmosferica_2_pred,PressaoAtmosferica_2_repaired,PressaoAtmosferica_3_pred,PressaoAtmosferica_3_repaired,PressaoAtmosferica_0_pred,PressaoAtmosferica_0_repaired,VelocidadeDoVento_3_pred,VelocidadeDoVento_3_repaired,PressaoAtmosferica_4_pred,PressaoAtmosferica_4_repaired,VelocidadeDoVento_2_pred,VelocidadeDoVento_2_repaired,TemperaturaDoAr_3_pred,TemperaturaDoAr_3_repaired,TemperaturaDoAr_2_pred,TemperaturaDoAr_2_repaired,VelocidadeDoVento_0_pred,VelocidadeDoVento_0_repaired,TemperaturaDoAr_0_pred,TemperaturaDoAr_0_repaired,UmidadeRelativa_3_pred,UmidadeRelativa_3_repaired,PontoDeOrvalho_3_pred,PontoDeOrvalho_3_repaired,TemperaturaDoAr_4_pred,TemperaturaDoAr_4_repaired,TemperaturaInterna_3_pred,TemperaturaInterna_3_repaired,UmidadeRelativa_0_pred,UmidadeRelativa_0_repaired,TemperaturaInterna_2_pred,TemperaturaInterna_2_repaired,TemperaturaInterna_0_pred,TemperaturaInterna_0_repaired,PontoDeOrvalho_0_pred,PontoDeOrvalho_0_repaired,TemperaturaInterna_4_pred,TemperaturaInterna_4_repaired,TemperaturaDoAr_1_pred,TemperaturaDoAr_1_repaired,VelocidadeDoVento_4_pred,VelocidadeDoVento_4_repaired,PontoDeOrvalho_2_pred,PontoDeOrvalho_2_repaired,UmidadeRelativa_1_pred,UmidadeRelativa_1_repaired,PontoDeOrvalho_1_pred,PontoDeOrvalho_1_repaired,TemperaturaInterna_1_pred,TemperaturaInterna_1_repaired,UmidadeRelativa_2_pred,UmidadeRelativa_2_repaired,RadiacaoSolar_4_pred,RadiacaoSolar_4_repaired,DirecaoDoVento_4_pred,DirecaoDoVento_4_repaired,PontoDeOrvalho_4_pred,PontoDeOrvalho_4_repaired,DirecaoDoVento_1_pred,DirecaoDoVento_1_repaired,UmidadeRelativa_4_pred,UmidadeRelativa_4_repaired,PressaoAtmosferica_1_pred,PressaoAtmosferica_1_repaired,VelocidadeDoVento_1_pred,VelocidadeDoVento_1_repaired,SensacaoTermica_0_pred,SensacaoTermica_0_repaired,SensacaoTermica_2_pred,SensacaoTermica_2_repaired,SensacaoTermica_3_pred,SensacaoTermica_3_repaired,RadiacaoSolar_1_pred,RadiacaoSolar_1_repaired,SensacaoTermica_4_pred,SensacaoTermica_4_repaired,SensacaoTermica_1_pred,SensacaoTermica_1_repaired
0,2011-01-01 06:30:00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,8.0,False,113.0,False,149.0,False,8.0,False,8.0,False,163.0,False,917.2,False,924.8,False,918.6,False,1.2,False,0.0,False,2.8,False,19.7,False,19.1,False,1.1,False,19.5,Fal

In [96]:
df_ungrouped = merged.copy()

for i in range(5):
    for feature in features[1:-1]: # Tira Local e Precipitacao
        df_ungrouped[f'{feature}_{i}'] = repaired[f'{feature}_{i}_pred']

df_ungrouped.head()

,index,Data,Hora,Data_Hora,UmidadeRelativa_0,PressaoAtmosferica_0,TemperaturaDoAr_0,TemperaturaInterna_0,PontoDeOrvalho_0,SensacaoTermica_0,RadiacaoSolar_0,DirecaoDoVento_0,VelocidadeDoVento_0,Precipitacao_0,Local_0,UmidadeRelativa_1,PressaoAtmosferica_1,TemperaturaDoAr_1,TemperaturaInterna_1,PontoDeOrvalho_1,SensacaoTermica_1,RadiacaoSolar_1,DirecaoDoVento_1,VelocidadeDoVento_1,Precipitacao_1,Local_1,UmidadeRelativa_2,PressaoAtmosferica_2,TemperaturaDoAr_2,TemperaturaInterna_2,PontoDeOrvalho_2,SensacaoTermica_2,RadiacaoSolar_2,DirecaoDoVento_2,VelocidadeDoVento_2,Precipitacao_2,Local_2,UmidadeRelativa_3,PressaoAtmosferica_3,TemperaturaDoAr_3,TemperaturaInterna_3,PontoDeOrvalho_3,SensacaoTermica_3,RadiacaoSolar_3,DirecaoDoVento_3,VelocidadeDoVento_3,Precipitacao_3,Local_3,UmidadeRelativa_4,PressaoAtmosferica_4,TemperaturaDoAr_4,TemperaturaInterna_4,PontoDeOrvalho_4,SensacaoTermica_4,RadiacaoSolar_4,DirecaoDoVento_4,VelocidadeDoVento_4,Precipitacao_4,Local_4
0,26,01/01/11,06:30:00,2011-01-01 06:30:00,87.2,918.6,19.5,22.6,17.3,NaN,8.0,163.0,1.1,0.0,Camilopolis,89.2,918.7,19.1,22.6,17.053488,NaN,9.0,109.0,2.7,0.0,Erasmo,89.4,917.2,19.1,22.5,17.3,NaN,8.0,113.0,2.8,0.0,Paraiso,86.5,924.8,19.7,23.2,17.4,NaN,8.0,149.0,1.2,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
1,27,01/01/11,06:45:00,2011-01-01 06:45:00,86.6,918.7,19.5,22.5,17.2,NaN,18.0,160.0,1.4,0.0,Camilopolis,88.6,918.7,19.2,22.6,16.909018,NaN,16.0,127.0,2.0,0.0,Erasmo,89.1,917.2,19.1,22.5,17.3,NaN,16.0,154.0,1.6,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,16.0,162.0,1.8,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
2,28,01/01/11,07:00:00,2011-01-01 07:00:00,86.0,918.7,19.6,22.6,17.2,NaN,41.0,108.0,2.2,0.0,Camilopolis,88.1,918.7,19.3,22.7,16.868210,NaN,35.0,138.0,2.7,0.0,Erasmo,88.4,917.2,19.1,22.6,17.3,NaN,28.0,116.0,2.3,0.0,Paraiso,85.6,924.8,19.9,23.3,17.4,NaN,30.0,181.0,1.7,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
3,29,01/01/11,07:15:00,2011-01-01 07:15:00,85.4,918.7,19.7,22.6,17.2,NaN,62.0,125.0,2.3,0.0,Camilopolis,88.1,918.7,19.3,22.7,16.778673,NaN,50.0,135.0,1.4,0.0,Erasmo,89.3,917.3,19.0,22.7,17.2,NaN,50.0,128.0,2.5,0.0,Paraiso,84.4,924.9,20.0,23.3,17.2,NaN,53.0,151.0,1.5,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria
4,30,01/01/11,07:30:00,2011-01-01 07:30:00,84.7,918.8,19.8,22.7,17.1,NaN,82.0,146.0,1.8,0.0,Camilopolis,87.4,918.7,19.4,22.8,16.722065,NaN,88.0,139.0,3.7,0.0,Erasmo,88.6,917.3,19.0,22.7,17.1,NaN,77.0,120.0,1.6,0.0,Paraiso,84.2,924.9,19.8,23.4,17.1,NaN,66.0,176.0,3.0,0.0,RM,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,Vitoria


In [97]:
df_grouped = reverse_ohe(df_ungrouped, features, ignoredFeatures, 5, '_')
df_grouped.head()

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,SensacaoTermica,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao
106838,2011-01-01 00:00:00,Camilopolis,79.300000,917.9,20.2,22.9,16.300000,18.800000,128.0,77.00000,7.900000,0.0
106838,2011-01-01 00:00:00,Erasmo,79.100000,919.5,20.1,23.7,16.300000,0.000000,0.0,133.00000,4.400000,0.0
106838,2011-01-01 00:00:00,Vitoria,76.818108,919.6,20.7,26.4,18.755472,19.681828,0.0,139.04718,7.891948,0.0
106838,2011-01-01 00:00:00,RM,78.400000,925.6,21.1,25.1,17.000000,21.000000,202.0,148.00000,5.800000,0.0
106838,2011-01-01 00:00:00,Paraiso,85.080925,918.1,20.7,24.4,17.827150,18.400000,209.0,184.00000,14.000000,0.0


In [98]:
df_grouped['Data_Hora'] = pd.to_datetime(df_grouped['Data_Hora'])

In [99]:
df_cluster = df_grouped[['Data_Hora', 'Local', 'Precipitacao', 'UmidadeRelativa', 'RadiacaoSolar']].copy()

In [100]:
df_cluster.isna().sum()

Data_Hora              0
Local              11435
Precipitacao       11435
UmidadeRelativa    11186
RadiacaoSolar      10190
dtype: int64

In [101]:
df_cluster.dropna(inplace=True)

In [102]:
df_cluster.isna().sum()

Data_Hora          0
Local              0
Precipitacao       0
UmidadeRelativa    0
RadiacaoSolar      0
dtype: int64

In [103]:
df_cluster['Ano'] = df_cluster['Data_Hora'].dt.year
df_cluster['Mes'] = df_cluster['Data_Hora'].dt.month
df_cluster['Dia'] = df_cluster['Data_Hora'].dt.day

In [104]:
#df_cluster['Local'] = df_cluster['Local'].rank(method='dense', ascending=False).astype(int)

In [105]:
df_cluster = df_cluster.groupby(['Local', 'Ano', 'Mes', 'Dia']).sum().reset_index()

In [106]:
sc = MinMaxScaler(feature_range=(0,1))

In [107]:
df_norm = sc.fit_transform(df_cluster[['Precipitacao', 'UmidadeRelativa', 'RadiacaoSolar']])
df_norm

array([[2.52016129e-04, 6.80625807e-01, 6.01467312e-01],
       [3.36021505e-04, 5.32090919e-01, 7.59649259e-01],
       [1.17607527e-02, 5.37557228e-01, 7.57523858e-01],
       ...,
       [0.00000000e+00, 4.28826601e-01, 1.18420248e-03],
       [0.00000000e+00, 3.99745076e-01, 1.02052040e-03],
       [0.00000000e+00, 4.41866291e-01, 1.10209423e-03]])

In [108]:
cluster = KMeans(n_clusters=4, random_state=42).fit(df_norm)

In [121]:
df_cluster['Cluster'] = cluster.labels_
df_cluster

,Local,Ano,Mes,Dia,Precipitacao,UmidadeRelativa,RadiacaoSolar,Cluster
0,Camilopolis,2011,1,1,0.6,6773.500000,29950.000000,3
1,Camilopolis,2011,1,2,0.8,5295.300000,37839.000000,3
2,Camilopolis,2011,1,3,28.0,5349.700000,37733.000000,3
3,Camilopolis,2011,1,4,0.0,4204.800000,35635.000000,3
4,Camilopolis,2011,1,5,65.6,5881.121853,32902.000000,3
...,...,...,...,...,...,...,...,...
15783,Vitoria,2019,12,5,15.2,7431.262495,-4.101408,1
15784,Vitoria,2019,12,6,0.0,6328.326175,0.000000,1
15785,Vitoria,2019,12,7,0.0,4267.626871,12.111713,2
15786,Vitoria,2019,12,8,0.0,3978.211296,3.948405,2


In [115]:
df_cluster.groupby(['Cluster', 'Local'])[['Cluster']].count()

Cluster
Cluster Local               
0       Camilopolis     1201
        Erasmo           425
        Paraiso         1183
        RM              1238
        Vitoria         1132
1       Camilopolis     1068
        Erasmo           946
        Paraiso          939
        RM              1134
        Vitoria          898
2       Camilopolis      150
        Erasmo          1601
        Paraiso          348
        RM               137
        Vitoria          852
3       Camilopolis      755
        Erasmo           105
        Paraiso          712
        RM               673
        Vitoria          291

In [116]:
df_cluster.groupby(['Cluster', 'Mes'])[['Cluster']].count()

Cluster
Cluster Mes         
0       1        478
        2        390
        3        492
        4        451
        5        500
        6        495
        7        412
        8        414
        9        374
        10       383
        11       405
        12       385
1       1        377
        2        405
        3        450
        4        417
        5        364
        6        410
        7        480
        8        519
        9        473
        10       410
        11       367
        12       313
2       1        250
        2        230
        3        232
        4        264
        5        327
        6        267
        7        265
        8        267
        9        263
        10       240
        11       234
        12       249
3       1        270
        2        222
        3        198
        4        194
        5        185
        6        149
        7        163
        8        157
        9        203
        10       243
        11       235
        12       317

In [113]:
fig = px.bar(df_cluster.groupby(['Cluster', 'Local'])[['Mes']].count().reset_index(),
             x="Cluster", y="Mes", color="Local", barmode="group")
fig.show()

In [114]:
fig = px.bar(df_cluster.groupby(['Cluster', 'Local'])[['Mes']].count().reset_index(),
             x="Local", y="Mes", color="Cluster", barmode="group")
fig.show()

In [128]:
cols = [c for c in merged.columns if 'UmidadeRelativa' in c]
merged[cols].corr(method='spearman')

,UmidadeRelativa_0,UmidadeRelativa_1,UmidadeRelativa_2,UmidadeRelativa_3,UmidadeRelativa_4
UmidadeRelativa_0,1.000000,0.493403,0.232698,0.730910,0.111509
UmidadeRelativa_1,0.493403,1.000000,-0.057890,0.539617,-0.228910
UmidadeRelativa_2,0.232698,-0.057890,1.000000,0.206135,0.615032
UmidadeRelativa_3,0.730910,0.539617,0.206135,1.000000,0.024495
UmidadeRelativa_4,0.111509,-0.228910,0.615032,0.024495,1.000000


In [129]:
cols = [c for c in df_ungrouped.columns if 'UmidadeRelativa' in c]
df_ungrouped[cols].corr(method='spearman')

,UmidadeRelativa_0,UmidadeRelativa_1,UmidadeRelativa_2,UmidadeRelativa_3,UmidadeRelativa_4
UmidadeRelativa_0,1.000000,0.507392,0.594872,0.803270,0.407979
UmidadeRelativa_1,0.507392,1.000000,0.261825,0.551175,0.113227
UmidadeRelativa_2,0.594872,0.261825,1.000000,0.643285,0.621256
UmidadeRelativa_3,0.803270,0.551175,0.643285,1.000000,0.424431
UmidadeRelativa_4,0.407979,0.113227,0.621256,0.424431,1.000000


In [131]:
regions[[c for c in regions.columns if 'Data_Hora' not in c]].sum() / regions.shape[0] * 100

UmidadeRelativa_0_error       13.988734
UmidadeRelativa_1_error       33.413759
UmidadeRelativa_2_error       35.274748
UmidadeRelativa_3_error       11.163959
UmidadeRelativa_4_error       40.423288
PressaoAtmosferica_0_error     8.719886
PressaoAtmosferica_1_error    41.133272
PressaoAtmosferica_2_error     7.730720
PressaoAtmosferica_3_error     8.061321
PressaoAtmosferica_4_error     9.679386
TemperaturaDoAr_0_error       10.987946
TemperaturaDoAr_1_error       31.892599
TemperaturaDoAr_2_error       10.396622
TemperaturaDoAr_3_error       10.045915
TemperaturaDoAr_4_error       12.384183
TemperaturaInterna_0_error    14.359218
TemperaturaInterna_1_error    33.880819
TemperaturaInterna_2_error    14.285385
TemperaturaInterna_3_error    13.154486
TemperaturaInterna_4_error    15.376071
PontoDeOrvalho_0_error        14.980866
PontoDeOrvalho_1_error        33.527475
PontoDeOrvalho_2_error        33.066677
PontoDeOrvalho_3_error        11.408201
PontoDeOrvalho_4_error        39.051443


In [ ]:
#df_grouped.corr()

In [119]:
#df_cluster.sample(n=100000).plot.scatter('Mes', 'Precipitacao')